In [17]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [2]:
def scrape(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data.get('headline', soup.find('h1', class_='content--title').text)
    date = json_data.get('datePublished', re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', str(html)).group(1))
    date_m = json_data.get('dateModified', '')
    genre = json_data.get('genre', [])
    if genre == []:
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    return notyet

def get_link_nogenre(start=0):
    notyet = []
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    for dic in data:
        if dic['genre'] == []:
            notyet.append(dic['url'])
    return notyet  

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)
        
easy = pd.read_json('nhkwebeasy.json')

In [4]:
htmls = ['https://www3.nhk.or.jp/news/html/20191117/k10012180511000.html']

In [96]:
htmls = get_link(0); htmls

['https://www3.nhk.or.jp/news/html/20190120/k10011784611000.html',
 'https://www3.nhk.or.jp/news/html/20191010/k10012120321000.html',
 'https://www3.nhk.or.jp/news/html/20191017/k10012135411000.html']

In [15]:
htmls = get_link_nogenre(); htmls[:50]

[]

# scrape

In [9]:
driver = webdriver.Chrome()

In [14]:
for nhkurl in tqdm.tqdm(htmls[:]):
    driver.get(f'https://web.archive.org/web/2019*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(30)
    html = driver.page_source.encode('utf-8')
    dic = scrape(html)
    js(dic)




  0%|          | 0/281 [00:00<?, ?it/s]


  0%|          | 1/281 [01:25<6:36:34, 84.98s/it]


  1%|          | 2/281 [02:30<6:08:02, 79.15s/it]


  1%|          | 3/281 [03:46<6:02:39, 78.27s/it]


  1%|▏         | 4/281 [05:10<6:08:41, 79.86s/it]


  2%|▏         | 5/281 [06:32<6:09:54, 80.41s/it]


  2%|▏         | 6/281 [07:47<6:02:22, 79.07s/it]


  2%|▏         | 7/281 [09:01<5:54:08, 77.55s/it]


  3%|▎         | 8/281 [10:19<5:52:43, 77.52s/it]


  3%|▎         | 9/281 [11:35<5:49:44, 77.15s/it]


  4%|▎         | 10/281 [12:44<5:37:50, 74.80s/it]


  4%|▍         | 11/281 [14:05<5:43:53, 76.42s/it]


  4%|▍         | 12/281 [15:11<5:29:05, 73.40s/it]


  5%|▍         | 13/281 [16:25<5:29:07, 73.68s/it]


  5%|▍         | 14/281 [17:26<5:11:03, 69.90s/it]


  5%|▌         | 15/281 [18:32<5:04:01, 68.58s/it]


  6%|▌         | 16/281 [19:37<4:58:37, 67.61s/it]


  6%|▌         | 17/281 [20:47<4:59:36, 68.09s/it]


  6%|▋         | 18/281 [21:53<4:56:26, 67.63s/it]


  7%|▋    

In [94]:
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")
dic = scrape(html)
print(dic)
js(dic)

{'id': 'k10011846961000', 'title': '買い物弱者など解決へ ロボットがコンビニの商品配達する実験', 'article': '高齢化に伴う「買い物弱者」などの解決につなげようと、自動で動くロボットがコンビニの商品を配達する実験が神奈川県の大学で行われました。\nこれは大手コンビニチェーンの「ローソン」と自動運転技術を開発するベンチャー企業の「ＺＭＰ」などが行いました。実験は大学内で行われ、利用者がスマートフォンの専用のアプリを使って商品を注文すると、高さ、幅などが１メートルに満たない箱型のロボットが、仮設の店舗から商品を届けます。センサーやカメラが周りの状況を認識しているということで、人が近づくと自動的に停止するほか、人とすれ違った際には音声であいさつもしてくれます。両社では、こうしたロボットで高齢化に伴う買い物弱者やドライバーの人手不足といった課題の解決につなげたい考えです。また公道を移動できるよう国に対して法整備を働きかけていくことにしています。ローソンの牧野国嗣オープン・イノベーションセンター長は「時間がなかったり、体が不自由で店に行けなかったりするなど、さまざまなニーズに応えるため、実用化を進めていきたい」と話していました。', 'genre': ['ビジネス', '社会'], 'keywords': [], 'url': 'https://www3.nhk.or.jp/news/html/20190313/k10011846961000.html', 'datePublished': '2019-03-13T17:35:18+09:00', 'dateModified': '2019-03-27T14:00:23+09:00'}


In [91]:
html = str(driver.page_source.encode('utf-8'))
re.search(r'"genre":(.+?),', html)

# clean category & keyword

In [75]:
with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print(len(data))

4712


In [103]:
# check category

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print(len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

4714


[('社会', 1287),
 ('国際', 1075),
 ('ビジネス', 642),
 ('スポーツ', 542),
 ('科学・文化', 537),
 ('政治', 527),
 ('暮らし', 319),
 ('地域', 252),
 ('気象・災害', 124)]

In [16]:
# genre <> keywords

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        j = re.sub('災害|気象', '気象・災害', j)
        if j in category:
            newgenre.append(j)
        else:
            newkey.append(j)
    for j in dic['keywords']:
        j = re.sub('災害|気象', '気象・災害', j)
        if j in category:
            newgenre.append(j)
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open('nhkweb.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    

In [22]:
normal = pd.read_json('nhkweb.json')
normal.id.value_counts()

k10012198091000    2
k10012197481000    2
k10012197851000    2
k10012261151000    1
k10011914981000    1
                  ..
k10011908771000    1
k10012235281000    1
k10012192751000    1
k10011908731000    1
k10012263711000    1
Name: id, Length: 4991, dtype: int64